In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在BigQuery数据上对TensorFlow模型进行训练

在Colab中打开

在Colab企业版中打开

在Workbench中打开

在GitHub上查看

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署一个用于在线预测的自定义表格分类模型。

了解更多关于[Vertex AI训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的信息。

### 目标

在这份笔记本中，你将学习如何使用 Vertex AI SDK for Python 在 Docker 容器中从 Python 脚本中创建一个自定义训练模型，然后通过发送数据来获取部署模型的预测。另外，您也可以使用 `gcloud` 命令行工具或在 Cloud 控制台上在线创建自定义训练模型。

本教程使用以下 Google Cloud ML 服务和资源：

- BigQuery
- Cloud Storage
- Vertex AI 管理的数据集
- Vertex AI 训练
- Vertex AI 端点

所执行的步骤包括：

- 创建一个 Vertex AI 自定义 `TrainingPipeline` 以训练模型。
- 训练一个 TensorFlow 模型。
- 部署 `Model` 资源到一个服务 `Endpoint` 资源。
- 进行预测。
- 取消部署 `Model` 资源。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的企鹅数据集。在本教程中，只使用数据集中的`culmen_length_mm`、`culmen_depth_mm`、`flipper_length_mm`和`body_mass_g`字段来预测企鹅的物种（`species`）。

### 成本

本教程使用 Google Cloud 的付费组件：

* Vertex AI
* Cloud Storage
* BigQuery

了解有关 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage
定价](https://cloud.google.com/storage/pricing)、[BigQuery 定价](https://cloud.google.com/bigquery/pricing) 的信息，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预计使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的包

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]'

重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等待完成后再继续下一步。⚠️</b>
</div>

### 在谷歌 Colab 上验证您的笔记本环境

在谷歌 Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息并初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建云存储桶

创建一个存储桶，用于存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有当您的存储桶不存在时，才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

导入Vertex AI Python SDK和其他必需的Python库。

In [ ]:
import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery

### 初始化用于Python的顶点 AI SDK

为您的项目和相应的存储桶初始化用于Python的顶点 AI SDK。

In [ ]:
# Initialize the Vertex AI SDK for Python
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 初始化BigQuery客户端

为您的项目初始化BigQuery Python客户端。

要使用BigQuery，请确保您的账号具有“BigQuery用户”角色。

In [ ]:
# Set up BigQuery client
bq_client = bigquery.Client(project=PROJECT_ID)

### 数据预处理和拆分数据
首先，您应该下载并预处理您的数据，用于训练和测试。

- 将分类特征转换为数字
- 删除未使用的列
- 删除无法使用的行
- 拆分训练和测试数据

In [ ]:
LABEL_COLUMN = "species"

# Define the BigQuery source dataset
BQ_SOURCE = "bigquery-public-data.ml_datasets.penguins"

# Define NA values
NA_VALUES = ["NA", "."]

# Download a table
table = bq_client.get_table(BQ_SOURCE)
df = bq_client.list_rows(table).to_dataframe()

# Drop unusable rows
df = df.replace(to_replace=NA_VALUES, value=np.NaN).dropna()

# Convert categorical columns to numeric
df["island"], _ = pd.factorize(df["island"])
df["species"], _ = pd.factorize(df["species"])
df["sex"], _ = pd.factorize(df["sex"])

# Split into a training and holdout dataset
df_train = df.sample(frac=0.8, random_state=100)
df_holdout = df[~df.index.isin(df_train.index)]

从BigQuery数据集创建一个Vertex AI表格数据集

从您的BigQuery训练数据中创建一个Vertex AI表格数据集资源。

在此处查看更多信息：https://cloud.google.com/vertex-ai/docs/training/using-managed-datasets

In [ ]:
# Create BigQuery dataset
bq_dataset_id = f"{PROJECT_ID}.dataset_id_unique"
bq_dataset = bigquery.Dataset(bq_dataset_id)
bq_client.create_dataset(bq_dataset, exists_ok=True)

In [ ]:
dataset = aiplatform.TabularDataset.create_from_dataframe(
    df_source=df_train,
    staging_path=f"bq://{bq_dataset_id}.table-unique",
    display_name="sample-penguins",
)

### 训练模型

有两种方式可以使用容器镜像来训练模型：

- **使用 Vertex AI 预构建的容器镜像**。如果使用预构建的训练容器镜像，还需指定一个要安装到容器镜像中的 Python 包。这个 Python 包包含了您的训练代码。

- **使用自定义的容器镜像**。如果使用自定义的容器，那么容器镜像必须包含您的训练代码。

本示范中将使用预构建的容器。

### 为训练脚本定义命令参数

准备要传递给训练脚本的命令行参数。
- `args`：传递给相应 Python 模块的命令行参数。在本例中，它们是：
  - `label_column`：要预测的数据中的标签列。
  - `epochs`：训练的时期数。
  - `batch_size`：训练的批量大小。

In [ ]:
JOB_NAME = "custom_job_unique"

EPOCHS = 20
BATCH_SIZE = 10

CMDARGS = [
    "--label_column=" + LABEL_COLUMN,
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
]

#### 训练脚本

在下一个单元格中，编写训练脚本`task.py`的内容。总的来说，脚本执行以下操作：

- 使用 BigQuery Python 客户端库从 BigQuery 表中加载数据。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分发策略。
- 根据参数`args.epochs`和`args.batch_size`训练模型(`fit()`)。
- 从环境变量`AIP_MODEL_DIR`中获取保存模型文件的目录。该变量是由训练服务设置的。
- 将训练完成的模型保存到模型目录中。

> **_注意:_** 为了提高模型性能，在训练之前建议对模型的输入进行归一化处理。请查看 TensorFlow 教程 [https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers#numerical_columns](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers#numerical_columns) 了解详细信息。

> **_注意:_** 以下训练代码要求您授予训练帐号“BigQuery Read Session User”角色。请查看 [https://cloud.google.com/vertex-ai/docs/general/access-control#service-agents](https://cloud.google.com/vertex-ai/docs/general/access-control#service-agents) 以了解如何找到该帐号。

In [ ]:
%%writefile task.py

import argparse
import numpy as np
import os

import pandas as pd
import tensorflow as tf

from google.cloud import bigquery
from google.cloud import storage

# Read environmental variables
training_data_uri = os.getenv("AIP_TRAINING_DATA_URI")
validation_data_uri = os.getenv("AIP_VALIDATION_DATA_URI")
test_data_uri = os.getenv("AIP_TEST_DATA_URI")

# Read args
parser = argparse.ArgumentParser()
parser.add_argument('--label_column', required=True, type=str)
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--batch_size', default=10, type=int)
args = parser.parse_args()

# Set up training variables
LABEL_COLUMN = args.label_column

# See https://cloud.google.com/vertex-ai/docs/workbench/managed/executor#explicit-project-selection for issues regarding permissions.
PROJECT_NUMBER = os.environ["CLOUD_ML_PROJECT_ID"]
bq_client = bigquery.Client(project=PROJECT_NUMBER)


# Download a table
def download_table(bq_table_uri: str):
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]
        
    # Download the BigQuery table as a dataframe
    # This requires the "BigQuery Read Session User" role on the custom training service account.
    table = bq_client.get_table(bq_table_uri)
    return bq_client.list_rows(table).to_dataframe()

# Download dataset splits
df_train = download_table(training_data_uri)
df_validation = download_table(validation_data_uri)
df_test = download_table(test_data_uri)

def convert_dataframe_to_dataset(
    df_train: pd.DataFrame,
    df_validation: pd.DataFrame,
):
    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)
    df_validation_x, df_validation_y = df_validation, df_validation.pop(LABEL_COLUMN)

    y_train = tf.convert_to_tensor(np.asarray(df_train_y).astype("float32"))
    y_validation = tf.convert_to_tensor(np.asarray(df_validation_y).astype("float32"))

    # Convert to numpy representation
    x_train = tf.convert_to_tensor(np.asarray(df_train_x).astype("float32"))
    x_test = tf.convert_to_tensor(np.asarray(df_validation_x).astype("float32"))

    # Convert to one-hot representation
    num_species = len(df_train_y.unique())
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_species)
    y_validation = tf.keras.utils.to_categorical(y_validation, num_classes=num_species)

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_validation = tf.data.Dataset.from_tensor_slices((x_test, y_validation))
    return (dataset_train, dataset_validation)

# Create datasets
dataset_train, dataset_validation = convert_dataframe_to_dataset(df_train, df_validation)

# Shuffle train set
dataset_train = dataset_train.shuffle(len(df_train))

def create_model(num_features):
    # Create model
    Dense = tf.keras.layers.Dense
    model = tf.keras.Sequential(
        [
            Dense(
                100,
                activation=tf.nn.relu,
                kernel_initializer="uniform",
                input_dim=num_features,
            ),
            Dense(75, activation=tf.nn.relu),
            Dense(50, activation=tf.nn.relu),            
            Dense(25, activation=tf.nn.relu),
            Dense(3, activation=tf.nn.softmax),
        ]
    )
    
    # Compile Keras model
    optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
    model.compile(
        loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer
    )
    
    return model

# Create the model
model = create_model(num_features=dataset_train._flat_shapes[0].dims[0].value)

# Set up datasets
dataset_train = dataset_train.batch(args.batch_size)
dataset_validation = dataset_validation.batch(args.batch_size)

# Train the model
model.fit(dataset_train, epochs=args.epochs, validation_data=dataset_validation)

tf.saved_model.save(model, os.getenv("AIP_MODEL_DIR"))

### 训练模型

在Vertex AI上定义您的自定义`TrainingPipeline`。

使用`CustomTrainingJob`类来定义`TrainingPipeline`。该类接受以下参数：

- `display_name`：此训练流程的用户定义名称。
- `script_path`：训练脚本的本地路径。
- `container_uri`：训练容器镜像的URI。
- `requirements`：脚本的Python包依赖项列表。
- `model_serving_container_image_uri`：可为模型提供预测的容器的URI，可以是预构建容器或自定义容器。

使用`run`函数开始训练。该函数接受以下参数：

- `args`：传递给Python脚本的命令行参数。
- `replica_count`：工作节点复制品的数量。
- `model_display_name`：如果脚本生成托管的`Model`，则为`Model`的展示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作节点复制品的加速器数量。

`run`函数创建一个训练流程，用于训练并创建一个`Model`对象。训练流程完成后，`run`函数会返回`Model`对象。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    requirements=["google-cloud-bigquery[pandas]", "protobuf<3.20.0"],
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
)

MODEL_DISPLAY_NAME = "penguins_model_unique"

# Start the training
model = job.run(
    dataset=dataset,
    model_display_name=MODEL_DISPLAY_NAME,
    bigquery_destination=f"bq://{PROJECT_ID}",
    args=CMDARGS,
)

### 部署模型

在使用模型进行预测之前，您必须将其部署到一个`终端点（Endpoint）`。您可以通过调用`Model`资源上的`deploy`函数来完成这个操作。这个操作会执行以下两件事情：

1. 为部署`Model`资源创建一个`终端点（Endpoint）`资源。
2. 将`Model`资源部署到`终端点（Endpoint）`资源中。

该函数接受以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：将流量分配到此模型的终端点的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，请指定`{ "0": 100 }`，其中 "0" 指的是正在上传的模型，100 表示 100% 的流量。
   - 如果有现有模型在终端点上，需要分配流量，则使用 `model_id` 指定`{ "0": 百分比, model_id: 百分比, ... }`，其中 `model_id` 是终端点上现有`DeployedModel`的ID。百分比必须总和为 100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点副本的加速器数。
- `starting_replica_count`：初始预留的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，只会预留一个实例。

### 流量分配

`traffic_split`参数被指定为一个Python字典。您可以将模型的多个实例部署到一个终端点，然后设置每个实例获得的流量百分比。

您可以使用流量分配逐渐将新模型引入生产环境。例如，如果您在生产中有一个现有模型占据了 100% 的流量，您可以将一个新模型部署到同一个终端点，将10% 的流量导向它，并将原始模型的流量减少到 90%。这样可以让您在最小化对大多数用户干扰的同时监视新模型的性能。

### 计算实例扩展

您可以指定单个实例（或节点）来提供您的在线预测请求。本教程使用单个节点，因此`MIN_NODES`和`MAX_NODES`变量均设置为`1`。

如果要使用多个节点来提供您的在线预测请求，请将`MAX_NODES`设置为您想要使用的最大节点数。Vertex AI会自动调整用于提供预测的节点数量，直至达到您设置的最大值。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本情况。

### 终端点

`deploy`方法会等到模型部署完成，并最终返回一个`终端点（Endpoint）`对象。如果这是第一次将模型部署到终端点，则可能需要额外几分钟来完成资源的预留。

In [ ]:
DEPLOYED_NAME = "penguins_deployed_unique"

endpoint = model.deploy(deployed_model_display_name=DEPLOYED_NAME)

发出一个在线预测请求，发送到您部署的模型。

### 准备测试数据

将其转换为Python列表，准备测试数据

In [ ]:
df_holdout_y = df_holdout.pop(LABEL_COLUMN)
df_holdout_x = df_holdout

# Convert to list representation
holdout_x = np.array(df_holdout_x).tolist()
holdout_y = np.array(df_holdout_y).astype("float32").tolist()

### 发送预测请求

现在您有了测试数据，可以使用它来发送预测请求。使用`Endpoint`对象的`predict`函数，该函数接受以下参数：

- `instances`：一组企鹅测量实例。根据您的自定义模型，每个实例应该是一个数字数组。您在上一步中准备了这个列表。

`predict`函数返回一个列表，列表中的每个元素对应请求中的一个实例。在每个预测的输出中，您将看到以下内容：

- 针对预测的置信水平(`predictions`)，在0到1之间，对应于十个类别中的每一个。

然后，您可以对预测结果进行快速评估：
1. `np.argmax`：将每个置信水平列表转换为标签
2. 打印预测结果

In [ ]:
predictions = endpoint.predict(instances=holdout_x)
y_predicted = np.argmax(predictions.predictions, axis=1)

y_predicted

### 撤销模型

要从服务端点资源中撤销所有`Model`资源，可以使用端点的`undeploy_all`方法。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除此教程中创建的个别资源：

- 训练作业
- 模型
- 终端
- 云存储桶

In [ ]:
# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

# Warning: Setting this to true deletes everything in your bucket
delete_bucket = True

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI